In [ ]:
import sqlite3
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
db = sqlite3.connect('./finalDupCompDatabase')
cursor = db.cursor()
nodeDivAge = {'Opisthokonta':1105/50,
             'Bilateria':824/50,
             'Chordata':684/50,
             'Vertebrata':615/50,
             'Gnathostomata':473/50,
             'Euteleostomi':435/50,
             'Sarcopterygii':413/50,
             'Tetrapoda':352/50,
             'Amniota':312/50,
             'Mammalia':177/50,
             'Theria':159/50,
             'Eutheria':105/50,
             'Boreoeutheria':96/50,
             'Euarchontoglires':90/50,
             'Primates':74/50,
             'Haplorrhini':67/50,
             'Simiiformes':43/50,
             'Catarrhini':29.4/50,
             'Hominoidea':20.2/50,
             'Hominidae':16.8/50,
             'Homininae':9.1/50,
             'Homo sapiens':6.7/50}

In [ ]:
# duplication node based
cursor.execute('''SELECT id, 
                        genLen, 
                        cdsLen,
                        evolRate, 
                        u_domains,
                        gc,
                        gc3,
                        domains,
                        specificity,
                        transCount,
                        intCount,
                        intAvg,
                        max_exp,
                        ess,
                        motif_number_1k,
                        PPIs,
                        intCov,
                        mis_Z_score,
                        pLI_score,
                        loftool_percentile,
                        s_het,
                        Phi,
                        RVIS,
                        EvoTol,
                        CAI,
                        IntDisProp,
                        dupCat_maj,
                        youngestNode,
                        oldestNode
                    FROM
                        gene_features
                    WHERE (dupCat_maj == "WGD" OR dupCat_maj == "SSD")
                     AND EvoTol != "Not expressed above threshold in this ontology"
                     AND CAI != "Error-check"
                     AND RVIS != "NA" 
                     AND pLI_score != "NA" ''')
res = [list(x) for x in cursor.fetchall()]
df = pd.DataFrame(res)
df.columns = ['id','gen_len','clen','evolRate','uDoms','gc','gc3',
              'doms','spec','trans_count','int_count','avg_int',
              'max_exp','ess','motifs','PPIs','int_cov','missenseZ','pLI',
              'loftool','shet','phi','rvis','EvoTol','CAI','IntDis','dup_type','minAge','maxAge']
df['minAge'] = [nodeDivAge[x] for x in df['minAge']]
df['maxAge'] = [nodeDivAge[x] for x in df['maxAge']]
df['ess'] = -(df['ess'])

In [ ]:
#model selection and diagnostics
modelDict = {}
featureList = []

for col in df.columns:
    noLog, noBox= False,False
    if col == 'id' or col == 'dup_type' or col == 'minAge' or col == 'maxAge':
        continue
    else:
        print(col)
        featureList.append(col)
        formula = col + '~ maxAge + dup_type'
        formula2 = col + '~ maxAge + dup_type + maxAge*dup_type'
        formula3 = 'np.log10(' + col + ') ~ maxAge + dup_type'
        formula4 = 'np.log10(' + col + ') ~ maxAge + dup_type + maxAge*dup_type'
        formula5 = 'stats.boxcox('+col + ')[0] ~ maxAge + dup_type'
        formula6 = 'stats.boxcox('+ col + ')[0] ~ maxAge + dup_type + maxAge*dup_type'

#         if col == 'shet':
#             data = df_shet
#         else:
#             
        data = df
        if min(df[col]) <= 0:
            df[col] = [x + (min([y for y in df[col] if y > 0]))/2 for x in df[col]]
        
        fittedModel = smf.ols(formula = formula, data = data).fit()
        fittedModel2 = smf.ols(formula = formula2, data = data).fit()
        try:
            fittedModel3 = smf.ols(formula = formula3, data=data).fit()
            fittedModel4 = smf.ols(formula = formula4, data=data).fit()
        except:
            fittedModel3 = None
            fittedModel4 = None
            noLog = True
            print('Log failed')
        try:
            fittedModel5 = smf.ols(formula = formula5, data=data).fit()
            fittedModel6 = smf.ols(formula = formula6, data=data).fit()
        except:
            fittedModel5 = None
            fittedModel6 = None
            noBox = True
            print('Boxcox failed')
       
        
        print()
        print('No changes')
        print(fittedModel.aic)
        print(sm.stats.jarque_bera(fittedModel.resid)[0])
        print('Interaction, no transform')
        print(fittedModel2.aic)
        print('Log transform')
        try:
            print(fittedModel3.aic)
            print(sm.stats.jarque_bera(fittedModel3.resid)[0])
            print('Interaction with log transform')
            print(fittedModel4.aic)
        except AttributeError:
            print()
        print('Box cox transform')
        try:
            print(fittedModel5.aic)
            print(sm.stats.jarque_bera(fittedModel5.resid)[0])
            print('Interaction, box cox transform')
            print(fittedModel6.aic)
        except AttributeError:
            print()
        print()
        print()
        if not noBox and not noLog: #all transforms available
            #check which transform is closest to normality
            for model in [fittedModel,fittedModel3,fittedModel5]:
                if sm.stats.jarque_bera(model.resid)[0] == min([sm.stats.jarque_bera(x.resid)[0] for x in [fittedModel,fittedModel3,fittedModel5]]):
                    #check aic to determine if an interaction term should be included
                    if model == fittedModel:
                        if model.aic < fittedModel2.aic:
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        elif model.aic - fittedModel2.aic < 2: #aic not significantly different, choose simpler model
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        else:
                            chosenModel = fittedModel2
                            modelDict[col] = chosenModel.model.formula
                    
                    elif model == fittedModel3:
                        if model.aic < fittedModel4.aic:
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        elif model.aic - fittedModel4.aic < 2: #aic not significantly different, choose simpler model
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        else:
                            chosenModel = fittedModel4
                            modelDict[col] = chosenModel.model.formula
                            
                    elif model == fittedModel5:
                        if model.aic < fittedModel6.aic:
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        elif model.aic - fittedModel6.aic < 2: #aic not significantly different, choose simpler model
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        else:
                            chosenModel = fittedModel6
                            modelDict[col] = chosenModel.model.formula
            
        elif noBox:
            for model in [fittedModel,fittedModel3]:
                if sm.stats.jarque_bera(model.resid)[0] == min([sm.stats.jarque_bera(x.resid)[0] for x in [fittedModel,fittedModel3]]):
                    #check aic to determine if an interaction term should be included
                    if model == fittedModel:
                        if model.aic < fittedModel2.aic:
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        elif model.aic - fittedModel2.aic < 2: #aic not significantly different, choose simpler model
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        else:
                            chosenModel = fittedModel2
                            modelDict[col] = chosenModel.model.formula
                    
                    elif model == fittedModel3:
                        if model.aic < fittedModel4.aic:
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        elif model.aic - fittedModel4.aic < 2: #aic not significantly different, choose simpler model
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        else:
                            chosenModel = fittedModel4
                            modelDict[col] = chosenModel.model.formula
                            

        elif noLog:
            for model in [fittedModel,fittedModel5]:
                if sm.stats.jarque_bera(model.resid)[0] == min([sm.stats.jarque_bera(x.resid)[0] for x in [fittedModel,fittedModel5]]):
                    #check aic to determine if an interaction term should be included
                    if model == fittedModel:
                        if model.aic < fittedModel2.aic:
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        elif model.aic - fittedModel2.aic < 2: #aic not significantly different, choose simpler model
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        else:
                            chosenModel = fittedModel2
                            modelDict[col] = chosenModel.model.formula
                    
                            
                    elif model == fittedModel5:
                        if model.aic < fittedModel6.aic:
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        elif model.aic - fittedModel6.aic < 2: #aic not significantly different, choose simpler model
                            chosenModel = model
                            modelDict[col] = chosenModel.model.formula
                        else:
                            chosenModel = fittedModel6
                            modelDict[col] = chosenModel.model.formula
        influence = chosenModel.get_influence()
        #studentized residuals (deleted residuals)
        studentized_residuals = influence.resid_studentized_external
        fig,ax = plt.subplots(1,3,figsize=(15,10))
        ax[0].plot(chosenModel.fittedvalues,studentized_residuals,'ko',ms=0.5)
        ax[0].set_ylabel('Standardised residuals')
        ax[0].set_xlabel('Fitted values')
        ax[1].hist(studentized_residuals)
        ax[1].set_xlabel('Standardised residuals')
        sm.graphics.qqplot(studentized_residuals,ax=ax[2])
        plt.title(formula)
        plt.savefig('REGRESSION_DIAG_PLOTS/'+col+'_oldest_post_model_selection.png',bbox_inches='tight')
        plt.show()
        plt.close()

In [ ]:
#megaplot of regressions
def scale_lightness(rgb, scale_l):
    import colorsys
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    return colorsys.hls_to_rgb(h, min(1, l * scale_l), s = s)
def formatPval(pval):
#     pval = (mannwhitneyu(list1,list2,alternative='two-sided').pvalue)*correction
#     if pval >= 0.05:
#         return 'ns'
    if pval >= 0.001:
        return round(pval,3)
    else:
        valList = '{:.2e}'.format(pval).split('e')
        return valList[0]+'x 10$^{-'+valList[1].lstrip('-').lstrip('0')+'}$'
fig, axesList = plt.subplots(10,5,figsize=(16,20),gridspec_kw={'hspace':0.7,'height_ratios':[1,0.25,1,0.07,1,0.07,1,0.25,1,0.07]})
axes = []
pAxes,lAxes=[],[]
for l in axesList[::2]:
    pAxes.extend(l)
for l in axesList[1::2]:
    lAxes.extend(l)
featureList = ['gen_len','clen','avg_int','int_count','int_cov',
              'CAI','IntDis','gc','gc3','doms',
              'uDoms','max_exp','spec','trans_count','motifs',
              'PPIs','evolRate','ess','missenseZ','pLI',
              'phi','rvis','shet','EvoTol','loftool']
namesDict = {'gen_len':('Genomic Length','log(bp)'),'clen':('CDS Length','log(bp)'),'int_count':('Intron Count','count'),
             'avg_int':('Mean intron length','log(bp)'),'int_cov':('Intron coverage','%'),'CAI':('Codon optimisation','CAI score'),
             'IntDis':('Intrinsic Disorder','%'),'gc':('GC content','%'),'gc3':('GC3 content','%'),
             'doms':('Total domains','count'),'uDoms':('Unique domains','count'),'max_exp':('Maximal expression','log(TPM)'),
             'spec':('Expression specificity','tau'),'trans_count':('Number of isoforms','count'),
             'motifs':('Number of regulatory motifs','count'),'PPIs':('PPIs','count'),'evolRate':('Rate of evolution','dN/dS'),
             'ess':('Cellular essentiality','-(CRISPR score)'),'missenseZ':('Missense Z score','score'),'pLI':('pLI','score'),
             'phi':('Phi','score'),'shet':('S$_{het}$','score'),'EvoTol':('EvoTol','score'),'loftool':('LoFTool','percentile'),
             'rvis':('RVIS','score')}
for col, ax, axL in zip(featureList,pAxes,lAxes):
    formula=modelDict[col]
    data = df
    fittedModel = smf.ols(formula = formula, data = data).fit()
    valsWGD = df[df['dup_type']=='WGD'][['maxAge',col]]
    valsSSD = df[df['dup_type']=='SSD'][['maxAge',col]]
    valsSSDLine = df[['maxAge',col]]
    valsSSDLine['dup_type'] = 'SSD'
    valsWGDLine = df[['maxAge',col]]
    valsWGDLine['dup_type'] = 'WGD'
    if col in ['evolRate','uDoms','gc','avg_int','ess','missenseZ','shet']: #log transform
        print(col)
        if min(df[col]) <= 0:
            df[col] = [x + (min([y for y in df[col] if y > 0]))/2 for x in df[col]]
        valsWGD[col] = np.log10(valsWGD[col])
        valsSSD[col] = np.log10(valsSSD[col])
    elif col in ['gen_len','clen','gc3','doms','trans_count','int_count','max_exp','int_cov',
                 'pLI','PPIs','motifs','rvis','EvoTol','CAI','IntDis']: #box cox transform
        if min(df[col]) <= 0:
            df[col] = [x + (min([y for y in df[col] if y > 0]))/2 for x in df[col]]
        #get lambda for total transform
        lam = stats.boxcox(df[col])[1]
        valsWGD[col] = stats.boxcox(valsWGD[col],lmbda=lam)
        valsSSD[col] = stats.boxcox(valsSSD[col],lmbda=lam)
    if col == 'shet':
        valsWGD = valsWGD[valsWGD[col]<-0.117]
        valsSSD = valsSSD[valsSSD[col]<-0.117]
    ax.set_fc((0,0,0,0.05))
    ax.grid(color='w',lw=2)
#     ax.set_yticks([])
    ax.set_xticks([])
    ax.plot(valsWGD['maxAge'],valsWGD[col],'o',color=scale_lightness((0.9,0.2,0),1.4),ms=0.7)
    ax.plot(valsWGDLine['maxAge'],fittedModel.predict(valsWGDLine),color=scale_lightness((0.9,0.2,0),1.4),lw=2,label='WGD')
    ax.plot(valsSSD['maxAge'],valsSSD[col],'o',color=scale_lightness((0.9,0.3,0),1.8),ms=0.7)
    ax.plot(valsSSDLine['maxAge'],fittedModel.predict(valsSSDLine),color=scale_lightness((0.9,0.3,0),1.8),lw=2,label='SSD')
    ax.set_title(namesDict[col][0])
    axL.axis('off')
    if col in ['gen_len','clen','evolRate','avg_int','ess','pLI','int_count','int_cov','PPIs','missenseZ']:
        axL.text(0.5,2.2,'Age:',ha='center',weight='bold',size=12)
        axL.text(0,1.65,'p='+str(formatPval(fittedModel.pvalues['maxAge'])) + ', coeff ='+str(round(fittedModel.params['maxAge'],3)),ha='left',size=12)
        axL.text(0.5,1.1,'Duplicate type:',ha='center',weight='bold',size=12)
        axL.text(0,0.55,'p='+str(formatPval(fittedModel.pvalues['dup_type[T.WGD]'])) + ', coeff ='+str(round(fittedModel.params['dup_type[T.WGD]'],3)),ha='left',size=12)
        if col in ['gen_len','clen','evolRate','avg_int','ess','pLI']:
            axL.text(0.5,0,'Age*Duplicate type:',ha='center',weight='bold',size=12)
            axL.text(0,-0.55,'p='+str(formatPval(fittedModel.pvalues['maxAge:dup_type[T.WGD]'])) + ', coeff ='+str(round(fittedModel.params['maxAge:dup_type[T.WGD]'],3)),ha='left',size=12)

    else:
        axL.text(0.5,5,'Age:',ha='center',weight='bold',size=12)
        axL.text(0,3,'p='+str(formatPval(fittedModel.pvalues['maxAge'])) + ', coeff ='+str(round(fittedModel.params['maxAge'],3)),ha='left',size=12)
        axL.text(0.5,1,'Duplicate type:',ha='center',weight='bold',size=12)
        axL.text(0,-1,'p='+str(formatPval(fittedModel.pvalues['dup_type[T.WGD]'])) + ', coeff ='+str(round(fittedModel.params['dup_type[T.WGD]'],3)),ha='left',size=12)
    
    if col == 'int_cov':
        ax.legend(loc=(0.6,0.71))
    
plt.savefig('totalRegressionModels.png',bbox_inches='tight')